In [23]:
import time
import yt_dlp
import random
import spotipy
from pygame import mixer 
from pprint import pprint
from datetime import datetime
from pydub import AudioSegment 
from ytmusicapi import YTMusic
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = "3a37e2adaf224237b96d901fd80b1829"
CLIENT_SECRET = "1c58b4a6904f451b8a205472a2a18f0e"
creds = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
ytmusic = YTMusic('oauth.json') 
sp = spotipy.Spotify(client_credentials_manager=creds)

with open('genres.txt', 'r') as f:
    genres = f.read().split('\n')

def get_random_song(genre=None, start_year=None, end_year=None):
    if not genre:
        genre = genres[random.randint(0,len(genres)-1)]
    if not end_year:
        datetime.now().year
    if not start_year:
        parameters = f"genre:{genre}"
    else:
        parameters = f"genre:{genre} year:{start_year}-{end_year}"

    results = sp.search(q=parameters, type="track", limit=1, offset=random.randint(0,1000))
    artist = results['tracks']['items'][0]['album']['artists'][0]['name']
    title = results['tracks']['items'][0]['name']
    query = f'{title} by {artist}'
    
    search_results = ytmusic.search(query, filter='songs', limit=1)
    id = search_results[0]['videoId']
    duration = search_results[0]['duration_seconds']
    return query, id, duration

def download_song(id):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl':'%(title)s',   
        'embedthumbnail': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192'
        }, {
            'key': 'FFmpegMetadata',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(f'https://www.youtube.com/watch?v={id}', download=True)

    filename = ydl.prepare_filename(info_dict)
    print(f"The downloaded file name is: {filename}")
    return filename

def create_segment(title, duration):
    song = AudioSegment.from_mp3(f"{title}.mp3") 
    start_time = random.randint(0,duration-5) * 1000
    segment = song[start_time:start_time+5000]
    segment.export(f"{title} (Clip).mp3", format="mp3") 

In [24]:
name, id, duration = get_random_song("Mexican Son", 1969, 2017)
title = download_song(id) 
create_segment(title, duration)

[youtube] Extracting URL: https://www.youtube.com/watch?v=SXnyxmBBFPk
[youtube] SXnyxmBBFPk: Downloading webpage
[youtube] SXnyxmBBFPk: Downloading ios player API JSON
[youtube] SXnyxmBBFPk: Downloading mweb player API JSON
[youtube] SXnyxmBBFPk: Downloading m3u8 information
[info] SXnyxmBBFPk: Downloading 1 format(s): 251
[download] Destination: El son de las heridas
[download] 100% of    4.36MiB in 00:00:02 at 1.60MiB/s   
[ExtractAudio] Destination: El son de las heridas.mp3
Deleting original file El son de las heridas (pass -k to keep)
[Metadata] Adding metadata to "El son de las heridas.mp3"
The downloaded file name is: El son de las heridas
